In [1]:
import os,sys

sys.path.append(os.path.join(sys.path[0], ".."))

from environment.GridEnv import GridWorld
from torchsummary import summary

import numpy as np
import os
import datetime
from collections import defaultdict
from itertools import chain
from typing import Dict, DefaultDict, List, Optional
from deap import creator, base, cma, tools
from agents.cma_model import Model

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def eaGenerateUpdate(toolbox, ngen, halloffame=None, stats=None, save_path=None,
                     verbose=__debug__):
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    for gen in range(ngen):
        # Generate a new population
        population = toolbox.generate()
        # Evaluate the individuals
        fitnesses = toolbox.map(toolbox.evaluate, population)
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit

        if halloffame is not None:
            halloffame.update(population)
            best_weights: np.ndarray = np.array(halloffame.items[0])
            np.save(save_path, best_weights)

        # Update the strategy with the evaluated individuals
        toolbox.update(population)

        record = stats.compile(population) if stats is not None else {}
        logbook.record(gen=gen, nevals=len(population), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook

In [3]:

NUM_PLAYERS: int = 1
LAMBDA: int = NUM_PLAYERS * 300
NGEN: int = 15000

SAVE_PATH: str = os.path.join('.', 'CMA', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"), 'weights.npy')
os.makedirs(SAVE_PATH[:SAVE_PATH.rfind(os.sep)], exist_ok=False)

def env_creator():
    return GridWorld(8)


def evaluate(individuals) -> List:
    env = env_creator()
    observation = env.reset()
    agent: str
    weights: Optional[np.ndarray]
    observation_space = env.observation_space
    act_space = env.action_space
    policy = Model.from_weights(observation_space, act_space.n, individuals)

    reward_sum: int=0
    done: bool = False
    while not done:
        action_probabilities: np.ndarray = policy(observation)
        action = np.argmax(action_probabilities)
        observation, reward, done, _ = env.step(action)
        reward_sum+=reward
    return [(reward_sum,)]

def map_to_population(evaluate_func, population):
    return chain.from_iterable(map(evaluate_func, population))


def train():
    env = env_creator()

    observation_space = env.observation_space
    act_space = env.action_space
    temp_model: Model = Model(observation_space, act_space.n)
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)
    toolbox = base.Toolbox()
    strategy=cma.Strategy(centroid=[5.0]*temp_model.num_parameters, sigma=5.0, lambda_=LAMBDA)
    toolbox.register("generate", strategy.generate, creator.Individual)
    toolbox.register("update", strategy.update)
    toolbox.register("evaluate", lambda ind: evaluate(ind))
    toolbox.register("map", map_to_population)

    del temp_model

    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    eaGenerateUpdate(toolbox, ngen=NGEN, stats=stats, halloffame=hof, save_path=SAVE_PATH) #Removed save path (IT WAS IN CUSTOM EA UPDATE)


if __name__ == '__main__':
    train()

gen	nevals	avg     	std   	min	max
0  	300   	-16.8544	7.9472	-40	0  
1  	300   	-15.7969	7.83412	-43.75	0  
2  	300   	-16.2196	7.92643	-40.5625	0  
3  	300   	-15.3269	7.82905	-43.6875	0  
4  	300   	-15.9256	8.05343	-43.5625	0  
5  	300   	-16.4219	7.56882	-40     	-3.125
6  	300   	-15.674 	8.03219	-39.1875	0     
7  	300   	-15.4006	7.4318 	-40.625 	-3.125
8  	300   	-15.7675	10.3007	-40.625 	99.5  
9  	300   	-16.7694	8.01182	-40.5625	0     
10 	300   	-15.6621	7.26372	-42.4375	0     
11 	300   	-16.5602	7.86101	-37.5   	0     
12 	300   	-17.3867	8.4623 	-43.75  	0     
13 	300   	-16.0254	7.2895 	-37.5   	-3.125
14 	300   	-16.0808	7.56938	-42.8125	0     
15 	300   	-16.3083	8.04709	-40.25  	0     
16 	300   	-15.7196	8.03929	-40.625 	0     
17 	300   	-16.8346	8.30341	-40.4375	0     
18 	300   	-16.1435	7.50936	-36.5625	-3.125
19 	300   	-15.8742	7.94949	-40.5625	0     
20 	300   	-16.1196	7.48188	-37.4375	0     
21 	300   	-16.1975	8.00776	-40.625 	0     
22 	300   	-16.3879	

186	300   	-16.1   	10.6612	-40.625 	99.5   
187	300   	-16.5958	8.42706	-40.4375	0      
188	300   	-15.984 	7.70126	-40.5625	-3.125 
189	300   	-16.194 	8.14288	-37.9375	-3.125 
190	300   	-17.0437	8.76815	-43.5625	0      
191	300   	-16.4898	7.74019	-43.75  	-3.125 
192	300   	-17.2502	8.03274	-40.625 	0      
193	300   	-16.07  	7.56842	-37.5   	0      
194	300   	-16.3467	8.61513	-40.625 	0      
195	300   	-16.3783	8.17939	-40.25  	0      
196	300   	-15.2927	8.08337	-37.5   	0      
197	300   	-16.1819	8.15486	-40.625 	0      
198	300   	-15.8579	7.677  	-37.5   	0      
199	300   	-16.704 	8.04219	-40.625 	-3.125 
200	300   	-16.0415	7.91751	-43.375 	0      
201	300   	-15.8879	8.24639	-37.5   	0      
202	300   	-16.4833	7.66235	-37.125 	0      
203	300   	-16.1373	7.67677	-40.5625	0      
204	300   	-16.2815	7.58235	-40.625 	0      
205	300   	-16.1029	7.85978	-40.625 	0      
206	300   	-16.276 	8.27148	-40.625 	0      
207	300   	-16.1887	8.08073	-43.75  	-3.125 
208	300   

369	300   	-16.226 	7.73864	-40.4375	0      
370	300   	-16.3342	8.11573	-37.5   	-3.125 
371	300   	-16.6608	8.44706	-40     	-3.125 
372	300   	-15.736 	8.01708	-40     	0      
373	300   	-15.8594	7.32873	-37.5   	0      
374	300   	-16.6921	8.09616	-40.625 	-3.125 
375	300   	-15.5258	7.63385	-40.5625	0      
376	300   	-16.0038	7.93974	-37.5   	-3.125 
377	300   	-15.7031	8.15768	-40.625 	0      
378	300   	-16.3344	7.87209	-40.625 	-3.125 
379	300   	-17.1108	8.05054	-41.8125	0      
380	300   	-15.9129	8.00417	-40     	0      
381	300   	-16.349 	7.88639	-37.5   	0      
382	300   	-16.7512	8.10406	-43.75  	0      
383	300   	-16.6233	8.3887 	-40.5625	0      
384	300   	-16.2321	7.56131	-34.375 	0      
385	300   	-16.8431	8.56435	-43.75  	0      
386	300   	-15.9246	8.05095	-43.375 	0      
387	300   	-15.8692	7.34678	-37.5   	0      
388	300   	-16.3912	7.55696	-37.5625	0      
389	300   	-16.3992	7.99311	-43.75  	0      
390	300   	-16.3725	7.90605	-39.8125	0      
391	300   

552	300   	-15.7035	7.59607	-40.5625	0      
553	300   	-15.9556	7.96203	-43.75  	-3.125 
554	300   	-16.0571	7.67988	-37.4375	0      
555	300   	-16.9817	8.40443	-40.625 	0      
556	300   	-16.3956	8.10593	-40.625 	0      
557	300   	-15.8715	7.54485	-40.625 	0      
558	300   	-15.8688	7.83032	-37.5   	0      
559	300   	-16.3535	7.97159	-43.375 	-3.125 
560	300   	-16.1021	7.83502	-40.625 	0      
561	300   	-15.5358	7.76303	-37.4375	-3.125 
562	300   	-15.591 	8.0541 	-40.25  	-3.125 
563	300   	-15.1894	7.54363	-36.875 	-3.125 
564	300   	-16.1875	7.58391	-43.5625	-3.125 
565	300   	-16.2444	8.06956	-40.625 	-3.125 
566	300   	-15.6   	7.20451	-39.125 	0      
567	300   	-16.1046	7.91445	-43.375 	0      
568	300   	-15.6306	8.02496	-39.3125	0      
569	300   	-15.4162	7.74688	-40.625 	0      
570	300   	-16.0308	7.82638	-37.5   	0      
571	300   	-16.4315	7.93851	-43.5625	0      
572	300   	-15.7977	7.9081 	-37.5   	0      
573	300   	-15.8394	10.2985	-40.4375	98.8125
574	300   

735	300   	-14.8604	7.90087	-40.625 	0      
736	300   	-15.8958	7.65856	-42.4375	0      
737	300   	-16.8694	7.46463	-40.625 	-3.125 
738	300   	-16.1612	7.62704	-43.375 	-3.125 
739	300   	-15.8044	8.13259	-37.5   	0      
740	300   	-16.0527	7.87478	-39.3125	0      
741	300   	-16.1123	8.13336	-40.625 	0      
742	300   	-16.6423	8.12301	-43.6875	0      
743	300   	-16.0871	8.22313	-43.5625	-3.125 
744	300   	-15.0279	7.97796	-37.5   	0      
745	300   	-15.8879	8.007  	-43.6875	-3.125 
746	300   	-16.485 	8.20765	-37.5   	0      
747	300   	-15.8571	7.59045	-40.625 	0      
748	300   	-16.9817	8.32682	-40.5625	0      
749	300   	-15.665 	7.98798	-42.8125	0      
750	300   	-15.2387	8.14932	-43.375 	0      
751	300   	-15.5412	8.01463	-42.75  	0      
752	300   	-16.5548	8.46237	-40.5625	0      
753	300   	-15.3969	8.00239	-40.4375	0      
754	300   	-16.499 	7.6051 	-39.3125	-3.125 
755	300   	-16.1265	7.62551	-40.5625	0      
756	300   	-16.5873	8.33355	-43.6875	-3.125 
757	300   

...

5931	300   	-12.37  	6.24842	-32.6875	0      
5932	300   	-12.4425	6.85981	-34.375 	0      
5933	300   	-14.0604	6.98427	-37.5   	0      
5934	300   	-12.5915	6.61962	-35.9375	-3.125 
5935	300   	-12.9135	6.39983	-32.8125	0      
5936	300   	-13.2094	6.74763	-34.375 	-3.125 
5937	300   	-13.4808	7.07492	-36.875 	0      
5938	300   	-12.5752	9.5182 	-37.5   	99.4375
5939	300   	-13.2192	6.93895	-37.125 	0      
5940	300   	-12.4965	7.02873	-40.625 	0      
5941	300   	-12.86  	6.83779	-35.1875	0      
5942	300   	-13.7513	7.13558	-42.1875	0      
5943	300   	-13.4206	7.02702	-40.625 	-3.125 
5944	300   	-12.8867	7.17781	-40.25  	0      
5945	300   	-13.2127	7.60734	-40.625 	0      
5946	300   	-13.0875	6.8534 	-37.5   	0      
5947	300   	-13.4202	6.98491	-37.1875	0      
5948	300   	-12.6256	6.85876	-34.3125	0      
5949	300   	-13.2627	7.21472	-34.375 	0      
5950	300   	-12.9746	6.85296	-41.9375	0      
5951	300   	-13.129 	6.80713	-39.0625	0      
5952	300   	-13.1229	7.07986	-39.6

6110	300   	-13.2665	6.63934	-34.375 	0      
6111	300   	-11.9725	6.21383	-34.375 	0      
6112	300   	-12.5908	6.28157	-43.75  	-3.125 
6113	300   	-13.0504	6.66326	-37.125 	0      
6114	300   	-12.1538	5.91952	-32.6875	0      
6115	300   	-12.765 	6.65808	-42.8125	0      
6116	300   	-13.0444	7.39887	-40.625 	-3.125 
6117	300   	-12.6735	6.46997	-35.8125	-3.125 
6118	300   	-13.0583	6.6799 	-37.5   	-3.125 
6119	300   	-13.2894	7.24706	-39.0625	0      
6120	300   	-11.8904	6.32014	-37.5   	0      
6121	300   	-12.8377	6.54204	-34     	0      
6122	300   	-13.0225	6.75299	-40.625 	0      
6123	300   	-12.1504	5.91454	-37.5   	0      
6124	300   	-12.1185	6.20441	-37.5   	0      
6125	300   	-11.8383	5.80032	-31.25  	0      
6126	300   	-12.7504	6.28456	-35.9375	-3.5   
6127	300   	-12.5823	6.00659	-33.1875	-3.125 
6128	300   	-12.0535	6.21433	-37.5   	0      
6129	300   	-12.3019	6.204  	-30.75  	0      
6130	300   	-12.6383	6.51019	-42.1875	0      
6131	300   	-13.2504	6.57858	-35.4

6289	300   	-13.1179	6.62077	-32.8125	-3.125 
6290	300   	-12.2802	6.25386	-34.375 	0      
6291	300   	-11.8504	8.90622	-37.5   	99.625 
6292	300   	-12.4219	6.21591	-34.1875	-3.125 
6293	300   	-13.2896	6.56809	-34.375 	-3.125 
6294	300   	-12.2646	6.17556	-35.4375	0      
6295	300   	-12.8285	6.80331	-40.625 	0      
6296	300   	-12.5244	6.29166	-34     	-3.125 
6297	300   	-13.0825	6.58833	-33.1875	0      
6298	300   	-12.9298	6.79538	-37.5   	-3.125 
6299	300   	-13.3483	9.24782	-38.5625	97.875 
6300	300   	-12.8202	6.32076	-34.375 	-3.125 
6301	300   	-12.881 	6.66272	-33.4375	0      
6302	300   	-12.9258	6.58844	-40     	-3.125 
6303	300   	-13.2746	7.22097	-38.5625	-3.125 
6304	300   	-13.0046	6.68004	-37.3125	-3.125 
6305	300   	-12.6613	6.77222	-32.8125	0      
6306	300   	-13.2296	6.60115	-34.375 	0      
6307	300   	-12.8883	6.71381	-34.375 	0      
6308	300   	-13.2169	6.88581	-40.4375	0      
6309	300   	-12.8169	6.34107	-34.375 	-3.125 
6310	300   	-12.4396	6.26824	-34.3

6468	300   	-12.0152	6.31147	-35.9375	0      
6469	300   	-12.8452	6.47129	-42.1875	0      
6470	300   	-12.2433	6.60308	-34.3125	0      
6471	300   	-13.3058	6.90462	-37.5   	-3.125 
6472	300   	-13.2081	6.84489	-43.6875	0      
6473	300   	-12.996 	6.91576	-37.125 	0      
6474	300   	-12.8915	6.4733 	-35.8125	-3.125 
6475	300   	-12.2662	6.16166	-35.9375	-3.125 
6476	300   	-13.02  	7.17912	-40.625 	0      
6477	300   	-13.3085	6.97753	-35.9375	0      
6478	300   	-12.5915	6.51647	-37.5   	-3.125 
6479	300   	-13.0979	6.77001	-37.5   	-3.125 
6480	300   	-12.8454	6.31717	-34.375 	0      
6481	300   	-12.6102	6.23994	-35.8125	-3.125 
6482	300   	-13.1937	6.71244	-40.625 	0      
6483	300   	-12.9235	9.41951	-41.9375	99.625 
6484	300   	-13.0506	6.49733	-39.0625	-3.125 
6485	300   	-13.1465	6.20516	-37.5   	-3.125 
6486	300   	-12.2683	9.43265	-39.875 	99.5   
6487	300   	-13.1019	7.07522	-35.9375	0      
6488	300   	-12.8681	6.28671	-35.4375	0      
6489	300   	-12.6604	6.31768	-37.4

KeyboardInterrupt: 